In [ ]:
import os
import sys
import json
import spotipy
import webbrowser
import spotipy.util as util
from json.decoder import JSONDecodeError
import pandas as pd

from app_creds import username, client_id, client_secret 

###app_creds.py should contain the following:
#this is your spotify id and the ids from a dev spotify api that you create
# username = ''
# client_id =''
# client_secret = ''


redirect_uri = 'http://localhost:7777/callback'
scope = 'user-follow-read'

token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)
token

In [ ]:
#Read in the top tracks csv
df = pd.read_csv('Toptrackslist.csv')

#Get the track id from the url
df['trackID'] = df.URL.apply(lambda x: x.split('/')[-1])

#setup spotipy function
sp = spotipy.Spotify(auth=token)

#produce a track df
track_df = pd.DataFrame(df.trackID).drop_duplicates(subset=['trackID']).reset_index(drop=True)
#get some basic track info
track_info = []
for i in track_df['trackID']:
    data = sp.track(i)
    output = {
        'trackID':i,
        'name':data['name'],
        'popularity':data['popularity'],
        'explicit':data['explicit'],
        'duration_ms':data['duration_ms'],
    }
    track_info.append(output)
#get track features    
track_feats = []
for i in track_df['trackID']:
    data = sp.audio_features(i)[0]
    output = {
        'trackID':i,
        'danceability': data['danceability'],
        'energy': data['energy'],
        'key': data['key'],
        'loudness': data['loudness'],
        'mode': data['mode'],
        'speechiness': data['speechiness'],
        'acousticness': data['acousticness'],
        'instrumentalness': data['instrumentalness'],
        'liveness': data['liveness'],
        'valence': data['valence'],
        'tempo': data['tempo'],
        'time_signature': data['time_signature']
    }
    track_feats.append(output)
    
track_infoDF = pd.DataFrame(track_info)
track_featsDF = pd.DataFrame(track_feats)

track_df = track_df.merge(track_infoDF,on='trackID')
track_df = track_df.merge(track_featsDF,on='trackID')


#first get all artist id. There can be more than one artist per track
artist_data = []
for i in track_df['trackID']:
    data = sp.track(i)['artists']
    for y in range(len(data)):
        x = data[y]
        z = {
            'artistID': x['id'],
            'name': x['name']
        }
        artist_data.append(z)
artist_df = pd.DataFrame(artist_data).drop_duplicates(subset=['artistID']).reset_index(drop=True)

#get additional artist data
artist_data = []
for i in artist_df['artistID']:
    data = sp.artist(i)
    info = {
        'artistID': data['id'],
        'followers': data['followers']['total'],
        'genres': data['genres'],
        'popularity': data['popularity']
    }
    artist_data.append(info)
infoDf = pd.DataFrame(artist_data)
artist_df = artist_df.merge(infoDf,on='artistID')


In [ ]:
#export csvs
track_df.to_csv('track_features.csv')
artist_df.to_csv('artists_details.csv')